# 安装Opencv

从配置文件创建虚拟环境并且激活

```bash
conda create -f envs.yaml
conda activate py_opencv
```

在 `notebook` 中选择 `python` 解释器，点一下顶上右边的选择解释器，也就是图中的 `py_opencv(Python 3.13.5)` 部分


![notebook head](images\notebookhead.png)


在弹出的菜单中选择 `Python环境` 

![notebook head](images\py_select_1.png)


在弹出的菜单中选择 `py_opencv` 环境

![notebook head](images\py_select_2.png)


# 计算机眼中的图片

## 图像的本质——像素、坐标与颜色

### 像素 (Pixel) - 图像的基本单元

“Pixel”是“Picture Element”（图像元素）的缩写。它是构成数字图像的最小、最基本的单位。你可以把它想象成一个无法再被分割的纯色小方块。我们所看到的任何一张高清、细节丰富的数码照片，放大、放大、再放大，最终看到的都会是这些纯色的小方块。

![pixels show](./images/weixin_screenshot.png)

这个图片里面的坐标就是当前鼠标对应的像素的坐标

**一张图像的核心属性，就是由所有像素的属性共同决定的。**

每个像素，都有两个核心信息：

1. 它的位置：它在这张马赛克拼图的哪个位置？
2. 它的值：它应该显示什么颜色？

其实还有一种图是`矢量图`，这种图没有像素的概念，类似于使用数学公式描述画面。所以其不受图像放大的影响，清晰度不会变。但是在导入opencv的进行处理的时候，都会转为一般的像素图片，所以我们这里不再多说。

### 坐标系 (Coordinate System) - 像素的位置

为了管理千千万万个像素的位置，计算机使用了一个二维坐标系。请注意，这个坐标系和我们初中数学学的直角坐标系有一个关键区别：

1. 原点 (0, 0) 在图像的左上角。
2. X 轴 水平向右延伸，代表图像的宽度 (Width)。
3. Y 轴 垂直向下延伸，代表图像的高度 (Height)。

举个例子，对于一张分辨率为 800 x 600 (宽x高) 的图片：

1. 左上角第一个像素的坐标是 (x=0, y=0)。
2. 右上角最后一个像素的坐标是 (x=799, y=0)。
3. 右下角最后一个像素的坐标是 (x=799, y=599)。
4. 图像正中心的像素坐标大约是 (x=400, y=300)。

但是，存储的时候又会反过来，所以在使用的时候，注意一下就可以了。

### 颜色与通道 (Color & Channels) - 定义像素的值

我们已经知道了像素的位置，那如何定义它的颜色呢？这就是“通道”概念的用武之地。

#### 最简单的情况：灰度图 (Grayscale Image)

一张黑白照片就是一张灰度图。它只有一个颜色通道。

+ 每个像素的值是一个单独的数字，用来表示其亮度。
+ 这个数字的范围通常是 0 到 255 (这被称为8位图像，因为 2^8 = 256)。
+ 0 代表纯黑色，255 代表纯白色，中间的数值则代表不同程度的灰色。

![pixels show](./images/grey_sample.png)

#### 彩色图像 (Color Image)

我们日常见到的大多数图片都是彩色的。计算机通过混合不同强度的三原色光来创造出成千上万种颜色。因此，一张彩色图通常有**三个颜色通道**。

+ 最常见的颜色模型是 RGB (Red, Green, Blue)。
+ ⚠️ 重要陷阱： OpenCV 默认使用的颜色通道顺序是 BGR (Blue, Green, Red)！ 这和很多其他软件（如Photoshop, Matplotlib）的RGB顺序不同。在进行颜色相关的操作时，一定要牢记这一点，否则你会得到奇怪的颜色。
+ 在BGR模型中，每个像素的值不再是一个数字，而是由 三个数字组成的元组 (Tuple) 或列表，分别代表该像素中蓝色、绿色和红色的强度。
+ 每个值的范围同样是 0 到 255。

#### 带有透明度的彩色图 (Four-Channel Image)

一张四通道图，通常被称为 **BGRA** 或 **RGBA** 图。

+ 前三个字母我们已经很熟悉了：B (Blue), G (Green), R (Red)。它们决定了像素的颜色。
+ 新增的第四个字母 A 代表 Alpha 通道。

**Alpha通道不控制颜色，而是控制像素的“不透明度 (Opacity)”或“透明度 (Transparency)”。**

和颜色通道一样，Alpha通道的值通常也是一个 0 到 255 的数字：

+ Alpha = 255 (完全不透明)：这是最“实心”的状态。像素会完全显示它自己的颜色，遮挡住它后面的一切。一张普通的JPG照片里所有像素的Alpha值都可以看作是255。
+ Alpha = 0 (完全透明)：像素是“隐形”的。它不显示任何颜色，像一块完全干净的玻璃，会直接显示出它后面的背景。
+ 0 < Alpha < 255 (半透明)：像素处于半透明状态，像一块有颜色的玻璃或一个水印。它会显示自己的颜色，但同时也会透出背景的颜色。Alpha的值越低，像素就越透明。

#### RGB-D图像-携带深度信息

**RGB-D** 代表 **Red, Green, Blue + Depth**。它不是一张单一的图像，而是一组同步并对齐的数据，通常包含两个部分：

+ 一张RGB（或BGR）图像： 这就是我们已经熟悉的标准彩色图像。它告诉我们场景中每个像素的颜色是什么。
+ 一张深度图 (Depth Map)： 这是新增的关键信息。它是一张单通道的“图像”，但它的像素值不是颜色或亮度，而是距离。

![](./images/RGBD-sample.jpg)

### 总结

**图像就是一个巨大的数字矩阵**

我们把所有知识点串联起来：

一张位图是一个由像素构成的网格（由坐标系定义），网格上每个点都有一个或多个数值来代表其颜色（由通道定义）。

这种 **“网格结构 + 数值”** 的组合，在数学和编程中有一个完美对应的概念——**矩阵 (Matrix)**，或者叫 **多维数组 (Multi-dimensional Array)**。

+ 一张灰度图，可以看作一个 二维矩阵。矩阵的形状是 (height, width)。矩阵中的每个元素就是该像素位置的灰度值（0-255）。
+ 一张彩色图，可以看作一个 三维矩阵。矩阵的形状是 (height, width, 3)。这里的 "3" 就代表B, G, R三个通道。你可以把它想象成3个二维矩阵叠在一起。



# OpenCV



> OpenCV 的全称是 Open Source Computer Vision Library，翻译过来就是“开源计算机视觉库”。

## 读取储存图片

### 读取图像

```python
image = cv2.imread(filepath, flags)
```
- `filepath`：图像文件的路径。
- `flags`: 一个可选参数，用于指定图片的读取方式。
    - cv2.IMREAD_COLOR (默认值): 以三通道BGR彩色图像格式读取。即使原始图片有透明度（Alpha通道），也会被忽略。
    - cv2.IMREAD_GRAYSCALE: 以单通道灰度图像格式读取。
    - cv2.IMREAD_UNCHANGED: 读取所有通道，包括Alpha通道（如果存在）。对于PNG等格式，这将返回一个四通道BGRA图像。

### 保存图像

```python
cv2.imwrite(filepath, image)
```

- filepath: 你希望保存的文件名和路径。文件的扩展名至关重要，例如：
    - '.jpg' 或 '.jpeg': 会以有损压缩的JPEG格式保存。适用于照片。
    - '.png': 会以无损压缩的PNG格式保存。适用于需要保留精确细节或透明度的图像。
- image: 你想要保存的图像矩阵。

### 显示图像

显示图像需要三个函数协同工作：`cv2.imshow()`, `cv2.waitKey()`, `cv2.destroyAllWindows()`。

#### `cv2.imshow(window_name, image)`
*   **功能**: 在一个窗口中显示图像。如果窗口不存在，则自动创建。
*   `window_name` (`str`): 窗口的标题和唯一标识符。
*   `image`: 要显示的图像矩阵。

#### `cv2.waitKey(delay)`
*   **功能**: 暂停程序并等待键盘输入，这是让窗口持续显示的关键。
*   `delay` (`int`): 等待的毫秒数。
    *   **`delay = 0`**: **无限等待**，直到用户按下任意键。用于显示单张图片。
    *   **`delay > 0`**: 等待指定毫秒数。用于播放视频流，例如 `cv2.waitKey(1)`。
*   **返回值**: 返回被按下的键的ASCII码，如果没有按键则返回`-1`。

#### `cv2.destroyAllWindows()`
*   **功能**: 关闭所有由OpenCV创建的窗口，释放资源。

In [1]:
import cv2

# --- 1. 读取 ---
img_path = './images/RGBD-sample.jpg'
image = cv2.imread(img_path)

# --- 2. 检查 ---
if image is None:
    print(f"错误: 无法在路径 '{img_path}' 找到图片。")
else:
    # --- 3. 显示 ---
    window_title = 'RobotMaster Demo'
    cv2.imshow(window_title, image)
    print(f"图片已在窗口 '{window_title}' 中显示。按任意键关闭。")
    
    # 等待用户交互
    cv2.waitKey(0)
    
    # 销毁窗口
    cv2.destroyAllWindows()
    print("窗口已关闭。")

    # --- 4. 处理与写入 ---
    # 将图像转为灰度
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # 保存灰度图像
    output_path = './images/RGBD-sample-gray.png'
    cv2.imwrite(output_path, gray_image)
    print(f"处理后的图像已保存到 '{output_path}'。")

图片已在窗口 'RobotMaster Demo' 中显示。按任意键关闭。
窗口已关闭。
处理后的图像已保存到 './images/RGBD-sample-gray.png'。


### 视频流

视频是一系列连续的图像帧。处理视频的核心是一个循环，在循环中对每一帧进行读取和处理。

#### 读取视频 (`cv2.VideoCapture`)

创建一个“视频捕捉对象”来从数据源获取视频帧。

**语法**
```python
cap = cv2.VideoCapture(source)
```
*   `source`: 数据源。
    *   **摄像头**: 整数 `0` (默认), `1`, `2`, ...
    *   **视频文件**: 字符串路径，如 `'my_video.mp4'`。

**常用方法**
*   `cap.isOpened()`: 检查视频源是否成功打开。
*   `ret, frame = cap.read()`: 读取下一帧。`ret` 为 `True` 表示成功，`frame` 是图像。
*   `cap.release()`: 释放视频源。

#### 写入视频 (`cv2.VideoWriter`)

创建一个“视频写入对象”来将一帧帧图像保存成视频文件。

**语法**
```python
writer = cv2.VideoWriter(filepath, fourcc, fps, frame_size)
```
**参数**
*   `filepath` (`str`): 输出文件名 (如 `'output.avi'`)。
*   `fourcc`: 指定视频编码器的4字节代码。常用 `cv2.VideoWriter_fourcc(*'XVID')`。
*   `fps` (`float`): 输出视频的帧率（每秒帧数）。
*   `frame_size` (`tuple`): `(width, height)`，视频分辨率。

**常用方法**
*   `writer.write(frame)`: 写入一帧。
*   `writer.release()`: 完成写入并释放资源。

#### 示例：从摄像头录制视频并保存

In [ ]:
import cv2

# --- 1. 创建读取和写入对象 ---
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("错误: 无法打开摄像头。")
    exit()

# 获取视频帧的尺寸
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
size = (w, h)

# 定义编码器和创建VideoWriter对象
fourcc = cv2.VideoWriter_fourcc(*'XVID')
writer = cv2.VideoWriter('output.avi', fourcc, 20.0, size)

print("正在录制... 按 'q' 键停止。")

# --- 2. 循环处理 ---
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # 将帧写入文件
    writer.write(frame)
    
    # 显示实时画面
    cv2.imshow('Recording...', frame)
    
    # 等待1ms，并检查退出键
    if cv2.waitKey(1) == ord('q'):
        break

# --- 3. 释放资源 ---
print("停止录制，正在保存文件...")
cap.release()
writer.release()
cv2.destroyAllWindows()
print("程序结束。")

## 图像处理

### 图像预处理 Image Preprocessing

#### 颜色空间转换 (`cv2.cvtColor`)

+ 目的： 简化计算，方便处理

+ 核心转换：
    + BGR → 灰度 (Grayscale): cv2.COLOR_BGR2GRAY，大多数分析算法的基础。

#### 图像平滑与滤波 (Image Smoothing & Filtering)

+ 目的：去除图像中的噪点（例如，传感器噪声），防止其干扰后续处理。
+ 核心方法：
    + 高斯模糊 (cv2.GaussianBlur): 最常用的滤波方式，通过加权平均来平滑图像。
+ 关键参数：核大小 (Kernel Size) 的概念及其对模糊程度的影响。

In [ ]:
# 导入OpenCV库
import cv2

# --- 步骤 1: 加载原始图像 ---
# 我们将使用这张图片作为所有预处理操作的起点。
print("正在加载原始图像...")
image_bgr = cv2.imread('./images/xiaogong.jpg')

# 健壮性检查：确保图像已成功加载
if image_bgr is None:
    print("错误：无法加载！请检查文件路径。")
    exit() # 如果图片没加载，后续操作无意义，直接退出程序

print("图像加载成功！")


# --- 步骤 2: 颜色空间转换 (`cv2.cvtColor`) ---
# 这是预处理中最常见的一步。

# 2.1 BGR -> 灰度 (Grayscale)
# 目的：将三通道的彩色信息压缩为单通道的亮度信息。
#       这可以大大减少计算量，并帮助算法专注于形状和纹理，而不是颜色。
print("正在将图像转换为灰度图...")
image_gray = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)


# --- 步骤 3: 图像平滑/模糊 (`cv2.GaussianBlur`) ---
# 目的：减少图像中的细节和噪声，这有助于防止后续的边缘检测或阈值处理
#       对无关紧要的噪点产生错误响应。

# 3.1 对原始彩色图进行高斯模糊
# (5, 5)是高斯核的大小，它必须是正奇数。核越大，图像就越模糊。
# 第三个参数是sigmaX，设为0表示由OpenCV根据核大小自动计算。
print("正在对原始彩色图进行高斯模糊...")
blurred_bgr = cv2.GaussianBlur(image_bgr, (5, 5), 0)


# 3.2 对灰度图进行高斯模糊 (这在实际应用中更常见)
print("正在对灰度图进行高斯模糊...")
blurred_gray = cv2.GaussianBlur(image_gray, (9, 9), 0) # 使用一个更大的核来观察效果


# --- 步骤 4: 显示所有处理结果进行对比 ---
# 使用cv2.imshow()将所有处理阶段的图像并排显示，以便直观地理解每个操作的效果。
print("正在显示所有结果图像...")

# 显示原始图像和其模糊版本
cv2.imshow('1. Original BGR', image_bgr)
cv2.imshow('2. Blurred BGR (5x5)', blurred_bgr)

# 显示颜色空间转换的结果
cv2.imshow('3. Grayscale', image_gray)

# 显示灰度图和其模糊版本
cv2.imshow('5. Blurred Grayscale (9x9)', blurred_gray)


# --- 步骤 5: 等待用户交互并清理 ---
print("\n所有窗口已显示。按任意键关闭所有窗口并退出程序。")
cv2.waitKey(0)  # 无限等待用户按键
cv2.destroyAllWindows() # 关闭所有打开的窗口
print("程序已结束。")

> ### **滤波原理详解：图像平滑的艺术**
> 
> #### **1. 核心思想：像素的“商量”与“妥协”**
> 
> 想象一张图片里有一个“噪点”——一个像素因为传感器错误，颜色值与周围格格不入（比如在一片黑色中突然出现一个纯白色的像素点）。这个噪点非常“突兀”，非常“尖锐”。
> 
> **滤波（或称平滑、模糊）的核心思想非常朴素：不要让一个像素“一意孤行”，它的最终颜色应该由它和它的“邻居们”一起“商量”决定。**
> 
> 通过取一个像素和它周围邻域像素值的**平均值**或**加权平均值**来替代该像素的原始值，就可以让那个突兀的噪点“妥协”，使其颜色值更接近于周围的像素，从而达到平滑图像、去除噪声的效果。
> 
> 这个“商量”的过程，在数学上被称为 **卷积 (Convolution)**。
> 
> #### **2. 关键工具：核 (Kernel)**
> 
> 为了实现“商量”，我们需要一个“规则模板”，这个模板决定了：
> 1.  “商量”的范围有多大？（是和周围一圈8个邻居商量，还是和两圈24个邻居商量？）
> 2.  每个邻居的“发言权”有多重？（是所有邻居都同等重要，还是离得近的邻居更重要？）
> 
> 这个“规则模板”，就叫做 **核 (Kernel)** 或 **卷积核 (Convolution Kernel)**。它本质上是一个**微型矩阵**。
> 
> #### **3. 工作流程：卷积 (Convolution) 的“滑动窗口”**
> 
> 卷积的过程就像一个“滑动窗口”在图像上移动。我们用一个简单的例子——**均值滤波 (Box Blur)**——来解释这个流程。
> 
> **假设我们有一个3x3的均值滤波核：**
> ```
>      1  1  1
> K = (1/9) * [ 1  1  1 ]
>      1  1  1
> ```
> 这个核的意思是：取3x3邻域内所有像素的值，求和，再除以9（取平均值）。所有邻居的“发言权”都是一样的。
> 
> **工作步骤：**
> 1.  **选择一个目标像素**：假设我们要计算图像中 `P5` 点的新像素值。
> 
> 2.  **对齐核的中心**：将核的中心（Anchor Point）与目标像素 `P5` 对齐。
> 
> 3.  **覆盖邻域**：此时，核会覆盖住 `P5` 和它周围的8个邻居，形成一个3x3的区域。
>     ```
>     原始图像像素值:          卷积核 K:
>     [ P1  P2  P3 ]         [ 1  1  1 ]
>     [ P4  P5  P6 ]         [ 1  1  1 ]
>     [ P7  P8  P9 ]         [ 1  1  1 ]
>     ```
> 
> 4.  **加权求和**：将图像邻域内的每个像素值与核中对应位置的权重相乘，然后将所有结果相加。
>     `Sum = (P1*1) + (P2*1) + ... + (P9*1)`
> 
> 5.  **计算新值**：将总和除以核的权重总和（这里是9）。
>     `New_P5_Value = Sum / 9`
> 
> 6.  **赋值**：用这个计算出的新值，来更新目标像素 `P5` 的值。
> 
> 7.  **滑动窗口**：将核向右移动一个像素，对齐新的目标像素，重复步骤2-6，直到遍历完图像中的所有像素。
> 
> 这个过程，就完成了对整张图像的一次均值滤波。
> 
> #### **4. 升级版：高斯滤波 (`cv2.GaussianBlur`)**
> 
> 均值滤波虽然简单，但有个缺点：它认为邻域内所有像素的“发言权”都一样，这显得有些“粗暴”。直觉上，**距离中心像素越近的邻居，应该有越大的发言权**。
> 
> **高斯滤波 (Gaussian Blur)** 就采纳了这个更合理的思想。它的核不是简单的全1矩阵，而是根据**高斯函数（正态分布/“钟形曲线”）**生成的一个权重矩阵。
> 
> **一个3x3的高斯核可能长这样：**
> ```
>       1  2  1
> K = (1/16) * [ 2  4  2 ]
>       1  2  1
> ```
> 
> *   **中心权重最高 (4)**：目标像素自身的权重最大。
> *   **距离越远，权重越低**：紧邻的像素权重为2，对角线的像素权重只有1。
> *   **权重总和**：`1+2+1+2+4+2+1+2+1 = 16`，所以最后需要除以16。
> 
> **高斯滤波的效果：**
> *   它产生的模糊效果更平滑、更自然，更好地保留了边缘信息，是实际应用中最常用的滤波方法。
> 
> #### **回到OpenCV的代码**
> 
> 现在，我们再来看`cv2.GaussianBlur`这个函数就豁然开朗了：
> 
> `cv2.GaussianBlur(image, (ksize_x, ksize_y), 0)`
> 
> *   `image`: 输入的图像矩阵。
> *   `(ksize_x, ksize_y)`: 这就是我们指定的**核的大小 (Kernel Size)**！例如 `(5, 5)`。
>     *   **核越大**，意味着“商量”的邻居范围越广，**图像就会越模糊**。
>     *   这就是为什么 `(9, 9)` 的模糊效果比 `(5, 5)` 更强烈。
> *   `0`: 这是标准差 `sigmaX`。设为0时，OpenCV会根据你提供的核大小自动计算出一个最合理的高斯分布。
> 
> **总结**
> 
> | 概念 | 解释 |
> | :--- | :--- |
> | **滤波/平滑** | 通过像素与邻居的“商量”来去除噪声、平滑图像。 |
> | **卷积** | 实现“商量”的数学过程，即“滑动窗口”式的加权求和。 |
> | **核 (Kernel)** | “商量”的规则模板，定义了邻域范围和每个邻居的发言权重。 |
> | **均值滤波** | 最简单的滤波，所有邻居权重相同。 |
> | **高斯滤波** | 更高级的滤波，遵循“离中心越近，权重越高”的原则，效果更自然。 |

### 图像分割与特征提取

目标：掌握将图像从复杂的多色世界简化为简单的黑白二值世界的方法。学习两种最核心的技术：阈值处理（用于分离物体）和边缘检测（用于勾勒物体轮廓）。

#### 图像阈值处理

**二值化**

+ 核心思想：设定一个“门槛”（阈值），然后遍历图像中的每一个像素。如果像素的亮度值高于这个门槛，就把它变成一种颜色（通常是白色）；如果低于门槛，就变成另一种颜色（通常是黑色）。
+ 输入：阈值处理函数的操作对象必须是单通道的灰度图。这是它工作的前提。
+ 输出：一张二值图像（Binary Image），即一张只包含两种颜色（黑和白）的图像。这张输出的图像通常被称为掩码 (Mask)。

```
retval, binary_image = cv2.threshold(grayscale_image, thresh_value, max_value, type)
```

*   `grayscale_image`: 输入的灰度图。
*   `thresh_value`: 你设定的“门槛”值，范围 `0-255`。
*   `max_value`: 当像素值超过门槛时，要赋予的新值，通常设为 `255` (白色)。
*   `type`: 阈值处理的类型，最常用的两个是：
    *   `cv2.THRESH_BINARY`: **标准二值化**。`像素值 > thresh_value` -> `max_value` (白), 否则 -> `0` (黑)。
    *   `cv2.THRESH_BINARY_INV`: **反向二值化**。`像素值 > thresh_value` -> `0` (黑), 否则 -> `max_value` (白)。

In [ ]:
import cv2

# --- 准备工作：加载图片并转为灰度图 ---
image_bgr = cv2.imread('images/ceiling.jpg')
if image_bgr is None:
    print("图片加载失败")
    exit()

# 转换为灰度图是阈值处理的前提
image_gray = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)
# 先进行模糊可以减少噪点对阈值分割的干扰
image_blur = cv2.GaussianBlur(image_gray, (5, 5), 0)


# --- 阈值处理 ---
# 设定一个阈值
thresh_value = 127

# 1. 标准二值化
# 所有亮度 > 127 的像素都变为 255 (白色)
ret, binary_img = cv2.threshold(image_blur, thresh_value, 255, cv2.THRESH_BINARY)

# 2. 反向二值化
# 所有亮度 > 127 的像素都变为 0 (黑色)
ret_inv, binary_inv_img = cv2.threshold(image_blur, thresh_value, 255, cv2.THRESH_BINARY_INV)


# --- 显示结果 ---
cv2.imshow('Original Image', image_bgr)
cv2.imshow('Blurred Grayscale', image_blur)
cv2.imshow('1. Binary Thresholding', binary_img)
cv2.imshow('2. Inverted Binary Thresholding', binary_inv_img)

cv2.waitKey(0)
cv2.destroyAllWindows()

**自适应阈值处理 (Adaptive Thresholding)**

我们之前使用的`cv2.threshold`方法，被称为**全局阈值处理**。它最大的特点是：**对整张图片，从左上角到右下角，都使用同一个“门槛”值**（例如，我们之前代码里用的`127`）。

这种“一刀切”的方法在一种情况下会彻底失效：**光照不均匀 (Uneven Illumination)**。

想象一张图片，左半边在明亮的灯光下，右半边在阴影里。
*   在明亮区域，可能大部分像素的亮度值都在`150`以上。
*   在阴影区域，可能大部分像素的亮度值都在`100`以下。

如果我们用全局阈值`127`：
*   明亮区域的所有东西都会变成白色。
*   阴影区域的所有东西都会变成黑色。
*   **结果：我们丢失了所有区域内的细节，分割彻底失败！**


自适应阈值处理的核心思想非常聪明：**不要用一个“门槛”衡量所有人，而是给每个人定一个他自己身边的“小标准”。**

它不再计算一个全局的阈值，而是**为图像中的每一个像素点，单独计算其最合适的阈值**。

**如何计算这个“小标准”？**
对于任何一个像素，它会看一看它周围的一个**邻域 (Neighborhood)**（比如一个 11x11 的小方块），然后根据这个小方块里的像素情况来决定当前像素的阈值。

```python
binary_image = cv2.adaptiveThreshold(grayscale_image, max_value, adaptive_method, threshold_type, block_size, C)
```

*   `grayscale_image`: 输入图像，**必须是单通道灰度图**。
*   `max_value`: 当像素满足条件时要赋予的新值，通常是`255`。
*   `adaptive_method`: 计算邻域阈值的方法，有两种：
    *   `cv2.ADAPTIVE_THRESH_MEAN_C`: **均值法**。阈值是邻域内所有像素的**平均值**，再减去一个常数`C`。
    *   `cv2.ADAPTIVE_THRESH_GAUSSIAN_C`: **高斯法**。阈值是邻域内像素的**高斯加权平均值**，再减去一个常数`C`。高斯加权意味着离中心点越近的像素“发言权”越大，这通常能得到比均值法更好的效果。
*   `threshold_type`: 和全局阈值一样，通常是 `cv2.THRESH_BINARY` 或 `cv2.THRESH_BINARY_INV`。
*   `block_size`: **非常重要的参数！** 它定义了计算阈值的邻域大小，例如`3`, `5`, `11`等。**这个值必须是正奇数！**
*   `C`: 一个常数。它是从计算出的平均值或加权平均值中**减去**的一个值。
    *   这个值的作用是进行微调。你可以把它理解为“宽容度”。
    *   如果`C`是正数，会使得最终的阈值变低，从而让更多的像素更容易变成白色（在`THRESH_BINARY`下）。
    *   如果`C`是负数，则反之。通常设为一个较小的正数。

In [3]:
import cv2

# --- 准备工作 ---
image_bgr = cv2.imread('./images/adaptiveThreshold_sample.jpg') # 建议换成一张光照不均的图片效果更明显
if image_bgr is None:
    print("图片加载失败")
    exit()

image_gray = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)
# 模糊对于自适应阈值同样有好处，可以使邻域计算更平滑
image_blur = cv2.GaussianBlur(image_gray, (5, 5), 0)


# --- 对比实验 ---

# 1. 全局阈值处理 (作为参照物)
ret, global_thresh = cv2.threshold(image_blur, 30, 255, cv2.THRESH_BINARY)


# 2. 自适应阈值处理 (均值法)
# blockSize = 11: 考虑像素周围11x11的邻域
# C = 2: 从计算出的均值中减去2，作为最终阈值
adaptive_mean_thresh = cv2.adaptiveThreshold(image_blur, 255, 
                                             cv2.ADAPTIVE_THRESH_MEAN_C, 
                                             cv2.THRESH_BINARY, 
                                             11, 2)

# 3. 自适应阈值处理 (高斯法)
# 参数与上面相同，只是方法换成了高斯加权
adaptive_gaussian_thresh = cv2.adaptiveThreshold(image_blur, 255, 
                                                 cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                                 cv2.THRESH_BINARY, 
                                                 11, 2)

# --- 显示结果 ---
cv2.imshow('Original Image', image_bgr)
cv2.imshow('1. Global Threshold (V=127)', global_thresh)
cv2.imshow('2. Adaptive Mean Threshold', adaptive_mean_thresh)
cv2.imshow('3. Adaptive Gaussian Threshold', adaptive_gaussian_thresh)

cv2.waitKey(0)
cv2.destroyAllWindows()

#### 边缘检测

**Canny**

阈值处理是基于“像素有多亮”来分割图像，而边缘检测则是基于“**像素亮度变化有多剧烈**”来寻找特征。物体的边界通常是图像中亮度发生急剧变化的地方。

Canny是一种非常强大和智能的边缘检测算法，它在内部执行了一系列复杂的步骤（模糊、计算梯度、抑制非边缘点、用双阈值连接边缘），以产生清晰、细致的边缘线。

*   **核心思想**：找出图像中亮度梯度最大的地方，并将它们连接成线。
*   **输入**：推荐使用**灰度图**（通常是经过轻微模糊处理的）。
*   **输出**：一张黑底白线的二值图像，白线就是检测到的边缘。

一般我们会对图像进行一次滤波，来平滑一下噪音

**语法**
```python
edges = cv2.Canny(image, threshold1, threshold2)
```

*   `image`: 输入的单通道图像。
*   `threshold1` (低阈值) & `threshold2` (高阈值): 这是Canny算法的精髓——**滞后阈值 (Hysteresis Thresholding)**。
    *   任何梯度高于 `threshold2` 的边缘点都会被直接认为是“强边缘”并保留下来。
    *   任何梯度低于 `threshold1` 的边缘点都会被抛弃。
    *   梯度在 `threshold1` 和 `threshold2` 之间的点，只有当它们**连接到一个“强边缘”**时才会被保留。
    *   这个机制使得Canny能够在保留弱边缘细节的同时，抑制噪声引发的伪边缘。
    *   **经验法则**：通常将`threshold2`设为`threshold1`的2倍或3倍，例如 `(50, 150)`。

In [4]:
import cv2

# --- 准备工作 ---
image_bgr = cv2.imread('./images/xiaogong.jpg')
if image_bgr is None:
    print("图片加载失败")
    exit()

image_gray = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)
# 对于Canny，一个较小的模糊核效果通常更好
image_blur = cv2.GaussianBlur(image_gray, (3, 3), 0)


# --- Canny边缘检测 ---
# 使用不同的阈值来观察效果
low_threshold = 50
high_threshold = 150
edges = cv2.Canny(image_blur, low_threshold, high_threshold)

# 尝试更严格的阈值
tighter_edges = cv2.Canny(image_blur, 100, 250)


# --- 显示结果 ---
cv2.imshow('Original Image', image_bgr)
cv2.imshow('Blurred Grayscale', image_blur)
cv2.imshow('Canny Edges (50, 150)', edges)
cv2.imshow('Tighter Canny Edges (100, 250)', tighter_edges)

cv2.waitKey(0)
cv2.destroyAllWindows()

#### 轮廓分析

**核心目标**: 在通过阈值处理或边缘检测得到一张黑白分明的二值图后，我们如何让程序去“看懂”图中那些白色的物体？轮廓分析就是解答这个问题的技术。它能帮助我们：
1.  **定位 (Locate)**：找到图中每一个独立物体的位置。
2.  **量化 (Quantify)**：用数字来描述这些物体的属性（大小、形状等）。
3.  **筛选 (Filter)**：根据这些数字属性，从一大堆物体中筛选出我们真正感兴趣的目标。

##### **轮廓的发现与可视化**

`cv2.findContours`
发现二值图像中的轮廓。

*   **函数签名 (Signature):**
    ```python
    contours, hierarchy = cv2.findContours(image, mode, method[, contours[, hierarchy[, offset]]])
    ```
*   **参数含义:**
    *   `image` (`numpy.ndarray`): 输入的单通道8位二值图像。**注意：此函数会修改输入的图像，如果想保留原二值图，请传入其副本 (`image.copy()`)**。
    *   `mode` (`int`): 轮廓的检索模式。
        *   `cv2.RETR_EXTERNAL`: 只检测最外层的轮廓。
        *   `cv2.RETR_LIST`: 检测所有轮廓，但不建立任何层级关系。
        *   `cv2.RETR_CCOMP`: 检测所有轮廓，并将它们组织成两级层次结构。
        *   `cv2.RETR_TREE`: 检测所有轮M廓，并重建完整的嵌套轮廓层次结构。
    *   `method` (`int`): 轮廓的近似方法。
        *   `cv2.CHAIN_APPROX_NONE`: 存储轮廓上所有的连续点。
        *   `cv2.CHAIN_APPROX_SIMPLE`: 压缩水平、垂直和对角线段，只保留其端点。**（推荐，节省内存）**
*   **返回值:**
    *   `contours` (`list`): 一个Python列表，其中每个元素都是一个独立的轮廓（一个NumPy数组，包含了该轮廓所有点的 `(x, y)` 坐标）。
    *   `hierarchy` (`numpy.ndarray`): 一个可选的输出数组，包含了轮廓的拓扑信息（层级关系）。

`cv2.drawContours`
在图像上绘制轮廓。

*   **函数签名 (Signature):**
    ```python
    image = cv2.drawContours(image, contours, contourIdx, color[, thickness[, lineType[, hierarchy[, maxLevel[, offset]]]]])
    ```
*   **参数含义:**
    *   `image` (`numpy.ndarray`): 目标图像，轮廓将被绘制在这张图上。
    *   `contours` (`list`): 轮廓列表，通常是 `cv2.findContours` 的输出。
    *   `contourIdx` (`int`): 指明要绘制哪个轮廓的索引。**如果为负数（通常是 `-1`），则绘制所有轮廓**。
    *   `color` (`tuple`): 轮廓的颜色，BGR格式，例如 `(0, 255, 0)` 代表绿色。
    *   `thickness` (`int`, 可选): 轮廓线的粗细。如果为负数（例如 `cv2.FILLED` 或 `-1`），则会填充轮廓内部。
*   **返回值:**
    *   `image` (`numpy.ndarray`): 绘制了轮廓的图像。

##### **轮廓的属性分析**

`cv2.contourArea`
计算轮廓的面积。

*   **函数签名 (Signature):**
    ```python
    area = cv2.contourArea(contour[, oriented])
    ```
*   **参数含义:**
    *   `contour` (`numpy.ndarray`): 单个轮廓的坐标点数组。
*   **返回值:**
    *   `area`: 轮廓所包围区域的面积。

`cv2.arcLength`
计算轮廓的周长（弧长）。

*   **函数签名 (Signature):**
    ```python
    perimeter = cv2.arcLength(curve, closed)
    ```
*   **参数含义:**
    *   `curve` (`numpy.ndarray`): 单个轮廓的坐标点数组。
    *   `closed` (`bool`): 一个布尔值，指明曲线是否是闭合的。对于从 `findContours` 得到的轮廓，这里**总是 `True`**。
*   **返回值:**
    *   `perimeter` (`float`): 轮廓的周长。

`cv2.boundingRect`
计算轮廓的垂直外接矩形。

*   **函数签名 (Signature):**
    ```python
    x, y, w, h = cv2.boundingRect(array)
    ```
*   **参数含义:**
    *   `array` (`numpy.ndarray`): 单个轮廓的坐标点数组。
*   **返回值:**
    *   `x, y, w, h` (`int`): 分别是矩形左上角的x坐标、y坐标，以及矩形的宽度和高度。
*   

`cv2.minAreaRect`
计算轮廓的最小旋转外接矩形。

*   **函数签名 (Signature):**
    ```python
    rotatedRect = cv2.minAreaRect(points)
    ```
*   **参数含义:**
    *   `points` (`numpy.ndarray`): 单个轮廓的坐标点数组。
*   **返回值:**
    *   `rotatedRect` (`RotatedRect`): 一个包含旋转矩形信息的对象，其内容为 `((center_x, center_y), (width, height), angle)`。
        *   要获取这个矩形的四个角点，通常需要配合 `cv2.boxPoints()` 函数。


`cv2.minEnclosingCircle`
计算轮廓的最小外接圆。

*   **函数签名 (Signature):**
    ```python
    center, radius = cv2.minEnclosingCircle(points)
    ```
*   **参数含义:**
    *   `points` (`numpy.ndarray`): 单个轮廓的坐标点数组。
*   **返回值:**
    *   `center` (`tuple`): `(float_x, float_y)`，圆心的浮点数坐标。
    *   `radius` (`float`): 圆的浮点数半径。


`cv2.minEnclosingTriangle`
计算并返回一个包围轮廓的、**面积最小**的三角形。

*   **函数签名 (Signature):**
    ```python
    retval, triangle = cv2.minEnclosingTriangle(points)
    ```
*   **参数含义:**
    *   `points` (`numpy.ndarray`): 输入的单通道、二维点集，通常就是我们的**单个轮廓**。

*   **返回值:**
    *   `retval` (`float`): 返回的最小三角形的**面积**。
    *   `triangle` (`numpy.ndarray`): 一个包含了三角形三个顶点坐标 `(x, y)` 的NumPy数组。它的形状会是 `(3, 1, 2)`，你需要对它进行适当的重塑或索引来获取这三个点。


`cv2.approxPolyDP`
用一个顶点数更少的多边形来逼近一个轮廓。

*   **函数签名 (Signature):**
    ```python
    approxCurve = cv2.approxPolyDP(curve, epsilon, closed[, approxCurve])
    ```
*   **参数含义:**
    *   `curve` (`numpy.ndarray`): 单个轮廓的坐标点数组。
    *   `epsilon` (`float`): 指定逼近精度的参数。这是原始轮廓与其近似多边形之间的最大距离。一个常用的方法是根据轮廓周长来设置它，例如 `0.04 * cv2.arcLength(curve, True)`。
    *   `closed` (`bool`): 指明逼近的曲线是否是闭合的。对于轮廓，这里**总是 `True`**。
*   **返回值:**
    *   `approxCurve` (`numpy.ndarray`): 近似多边形的顶点坐标数组。通过 `len(approxCurve)` 可以得到顶点数。

In [6]:
import cv2
import numpy as np

# --- 1. 准备工作: 加载图像 -> 灰度 -> 模糊 -> 二值化 ---
print("正在加载和预处理图像...")
image = cv2.imread('./images/counter_sample.png')
if image is None:
    print("错误：无法加载！请确认文件存在。")
    exit()

# 创建一个副本用于绘制，这样可以保留原始图像
output_image = image.copy()

# 标准预处理流程
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5, 5), 0)
# 我们希望物体是白色，背景是黑色，所以使用标准二值化
# 如果你的图片是白底黑图，则需要用THRESH_BINARY_INV
_, binary = cv2.threshold(blur, 50, 255, cv2.THRESH_BINARY)
# 为了演示方便，假设我们的shapes.png是黑底白图

print("预处理完成，正在寻找轮廓...")


# --- 2. 发现轮廓 ---
# cv2.RETR_EXTERNAL: 只寻找最外层的轮廓，非常适合分析独立物体。
# cv2.CHAIN_APPROX_SIMPLE: 压缩轮廓点，节省内存。
contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

print(f"发现了 {len(contours)} 个轮廓。现在开始分析每一个...")


# --- 3. 遍历、分析和筛选每一个轮廓 ---
for i, contour in enumerate(contours):
    
    # --- 3.1 计算基本属性 ---
    # 计算面积
    area = cv2.contourArea(contour)
    # 计算周长
    perimeter = cv2.arcLength(contour, True) # True表示轮廓是闭合的
    
    # --- 3.2 筛选1: 按面积过滤 ---
    # 这是非常重要的一步，可以过滤掉绝大多数由噪声产生的无用小轮廓
    if area < 100:
        continue # 如果面积太小，就跳过这个轮廓，不进行后续分析

    print(f"\n--- 分析轮廓 #{i+1} (面积: {int(area)}) ---")

    # --- 3.3 形状判断：使用多边形逼近 ---
    # epsilon的值是关键，它决定了近似的精度。值越小，越接近原始轮廓。
    # 0.01到0.05之间的系数是常用范围。
    epsilon = 0.04 * perimeter
    approx = cv2.approxPolyDP(contour, epsilon, True)
    num_vertices = len(approx) # 获取近似多边形的顶点数

    # 默认形状为未知
    shape_name = "Unknown"
    
    # --- 3.4 可视化：获取边界框并绘制 ---
    # 使用垂直外接矩形来定位
    x, y, w, h = cv2.boundingRect(approx)
    
    # --- 3.5 筛选2: 根据顶点数进行分类和绘制 ---
    if num_vertices == 3:
        shape_name = "Triangle"
        # 用绿色框出三角形
        cv2.rectangle(output_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    elif num_vertices == 4:
        # 对于四边形，可以再根据长宽比判断是正方形还是矩形
        aspect_ratio = float(w) / h
        if 0.95 <= aspect_ratio <= 1.05:
            shape_name = "Square"
        else:
            shape_name = "Rectangle"
        # 用蓝色框出四边形
        cv2.rectangle(output_image, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # 如果顶点数很多，我们可以认为是圆形
    elif num_vertices > 4:
        shape_name = "Circle"
        # 对于圆形，用最小外接圆来标记更合适
        (cx, cy), radius = cv2.minEnclosingCircle(contour)
        center = (int(cx), int(cy))
        radius = int(radius)
        # 用红色圆圈标记圆形
        cv2.circle(output_image, center, radius, (0, 0, 255), 2)
    
    print(f"判断形状为: {shape_name} ({num_vertices} 个顶点)")

    # --- 3.6 添加文本标签 ---
    # 在边界框的上方写上形状名称
    cv2.putText(output_image, shape_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)


# --- 4. 显示最终结果 ---
cv2.imshow('Original Image', image)
cv2.imshow('Binary Mask', binary)
cv2.imshow('Contour Analysis Result', output_image)

print("\n分析完成。按任意键退出。")
cv2.waitKey(0)
cv2.destroyAllWindows()

正在加载和预处理图像...
预处理完成，正在寻找轮廓...
发现了 4 个轮廓。现在开始分析每一个...

--- 分析轮廓 #1 (面积: 43460) ---
判断形状为: Rectangle (4 个顶点)

--- 分析轮廓 #2 (面积: 26985) ---
判断形状为: Triangle (3 个顶点)

--- 分析轮廓 #3 (面积: 44338) ---
判断形状为: Circle (8 个顶点)

--- 分析轮廓 #4 (面积: 42746) ---
判断形状为: Rectangle (4 个顶点)

分析完成。按任意键退出。


### 形态学操作

**核心目标**：学习如何使用一系列基于形状的图像处理技术来清理和优化二值图像。这些技术主要解决两个经典问题：
1.  **去除噪声**：消除二值图中孤立的、无意义的小像素点。
2.  **修复形状**：连接物体中断开的裂缝，或者填充物体内部的小孔洞。

**基础：结构元素 (Structuring Element) 或 核 (Kernel)**

所有形态学操作都依赖于一个叫做“**结构元素**”或“**核**”的东西。它和我们之前在滤波原理里讲的“核”很相似，也是一个微型矩阵，定义了操作的**邻域范围和形状**。

你可以把它想象成一个“画笔的笔刷”。
*   **笔刷的形状**：可以是矩形、圆形、十字形等。
*   **笔刷的大小**：例如`3x3`, `5x5`。

在OpenCV中，我们可以用 `cv2.getStructuringElement()` 来方便地创建一个核。

```python
# 创建一个 5x5 大小的矩形核
kernel_rect = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

# 创建一个 5x5 大小的圆形（椭圆）核
kernel_ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
```
通常情况下，**矩形核**已经能满足绝大多数需求。

#### 腐蚀与膨胀

##### **`cv2.erode` - 腐蚀**

*   **它做什么**：“腐蚀”掉物体（白色区域）边界的一层像素。
*   **如何理解**：想象一下，核（笔刷）在图像上滑动。只有当**核所覆盖的所有像素都为白色**时，核中心的那个像素才会被保留为白色；否则，它就会被“腐​​蚀”成黑色。
*   **结果**：
    1.  白色区域整体**变瘦**。
    2.  小的、孤立的白色噪点（小于核的大小）会**直接消失**。
    3.  物体之间的细小连接可能会**断开**。

**函数签名**
```python
eroded_image = cv2.erode(source_image, kernel, iterations=1)
```
*   `iterations`: 操作重复的次数。次数越多，腐蚀效果越强烈。

##### **`cv2.dilate` - 膨胀**

*   **它做什么**：在物体（白色区域）边界“生长”出一层像素。
*   **如何理解**：核在图像上滑动。只要核所覆盖的区域内**至少有一个像素是白色**的，核中心的那个像素就会被“膨胀”成白色。
*   **结果**：
    1.  白色区域整体**变胖**。
    2.  物体内部的黑色小孔洞会被**填充**掉。
    3.  两个原本分离但距离很近的物体可能会**连接**在一起。

**函数签名**
```python
dilated_image = cv2.dilate(source_image, kernel, iterations=1)
```

#### **开运算与闭运算 (`cv2.morphologyEx`)**

##### **开运算 (Opening)**

*   **定义**：**先腐蚀，后膨胀** (`Opening = Dilate(Erode(Image))`)。
*   **效果**：
    1.  第一步的**腐蚀**会消除掉所有小于核的孤立白色噪点。
    2.  第二步的**膨胀**会将那些被腐蚀“瘦身”了的、真正的大物体再“胖”回原来的大小。
*   **核心用途**：**去除小的、外部的“椒盐噪声”**（背景中的白色噪点）。它能有效地“打开”物体之间细小的连接，但基本不改变大物体的尺寸。

##### **闭运算 (Closing)**

*   **定义**：**先膨胀，后腐蚀** (`Closing = Erode(Dilate(Image))`)。
*   **效果**：
    1.  第一步的**膨胀**会填充掉物体内部的小孔洞，并连接邻近的物体。
    2.  第二步的**腐蚀**会将那些被膨胀“长胖”了的大物体再“瘦”回原来的大小。
*   **核心用途**：**填充物体内部的小孔洞或裂缝**。它能有效地“关闭”物体上的小缺口。

**函数签名**
```python
result_image = cv2.morphologyEx(source_image, operation_type, kernel)
```
*   `operation_type`: 你想执行的操作类型。
    *   `cv2.MORPH_OPEN`: 开运算
    *   `cv2.MORPH_CLOSE`: 闭运算
    *   还有其他如 `MORPH_GRADIENT`, `MORPH_TOPHAT` 等，但开闭运算最常用。

In [13]:
import cv2
import numpy as np

# --- 1. 创建一个用于演示的“不完美”的二值图像 ---
# 创建一个500x500的黑色背景
image = np.zeros((500, 500), dtype=np.uint8)

# 在中心画一个大的白色矩形，并在内部留一个小黑洞
cv2.rectangle(image, (100, 100), (400, 400), 255, -1) # 白色矩形

# 添加一些小黑色孔洞
for i in range(20):
    x = np.random.randint(0, 500)
    y = np.random.randint(0, 500)
    # 确保噪点不在大矩形内部，以模拟背景噪声
    if (100 < x < 400 and 100 < y < 400):
        cv2.circle(image, (x, y), 2, 0, -1)

# 人为地添加一些“椒盐噪声” (外部的白色小点)
for i in range(50):
    x = np.random.randint(0, 500)
    y = np.random.randint(0, 500)
    # 确保噪点不在大矩形内部，以模拟背景噪声
    if not (100 < x < 400 and 100 < y < 400):
        cv2.circle(image, (x, y), 2, 255, -1)

# 保存一份有问题的原始图副本
original_noisy = image.copy()


# --- 2. 定义结构元素 (核) ---
# 核的大小决定了操作的强度。
# 5x5的核对于修复我们创建的2像素半径的噪点和孔洞来说是足够的。
kernel = np.ones((5, 5), np.uint8) # 也可以用 getStructuringElement


# --- 3. 应用各种形态学操作 ---

# 腐蚀：物体变瘦，外部小噪点消失，但内部孔洞变大
eroded = cv2.erode(original_noisy, kernel, iterations=1)

# 膨胀：物体变胖，内部孔洞消失，但外部噪点也变大
dilated = cv2.dilate(original_noisy, kernel, iterations=1)

# 开运算：先腐蚀后膨胀
# 效果：有效地去除了外部的白色噪点，但内部的孔洞依然存在
opening = cv2.morphologyEx(original_noisy, cv2.MORPH_OPEN, kernel)

# 闭运算：先膨胀后腐蚀
# 效果：有效地填充了内部的黑色孔洞，但外部的噪点依然存在
closing = cv2.morphologyEx(original_noisy, cv2.MORPH_CLOSE, kernel)


# --- 4. 组合使用：先开后闭 ---
# 这是一个非常经典、强大的修复流程！
# 第一步：用开运算去除外部噪声
temp_img = cv2.morphologyEx(original_noisy, cv2.MORPH_OPEN, kernel)
# 第二步：在去噪后的结果上，用闭运算填充内部孔洞
perfect_img = cv2.morphologyEx(temp_img, cv2.MORPH_CLOSE, kernel)


# --- 5. 显示所有结果进行对比 ---
cv2.imshow('1. Original Noisy', original_noisy)
cv2.imshow('2. Erosion', eroded)
cv2.imshow('3. Dilation', dilated)
cv2.imshow('4. Opening', opening)
cv2.imshow('5. Closing', closing)
cv2.imshow('6. Perfect', perfect_img)

cv2.waitKey(0)
cv2.destroyAllWindows()

## 结语

今天讲的只是九牛一毛，更多操作还需要各位自己探索，可以查阅文档，问问AI，上网搜搜，接下来的也有作业的内容，需要大家完成。